# Analysing Topical Shifts in Migration Publication via Title Words

The titles of publications on migration management give us an overview of what the scientific community focused in their research over the decades.

The dataset consists of authors, titles and publication year of publications in the Research group on European Migration Problems series (REMP, since 1951), and the journals International Migration (IM, since 1964) and International Migration Review (IMR, since 1964), over the period 1951-2000. 

We use Content Analysis as a method to investigate topical shifts in the discourse around migration management. “Content analysis is an approach to the analysis of documents and texts (which may consist of words and/or images and may be printed or online, written or spoken) that seeks to quantify content in terms of predetermined categories and in a systematic and replicable manner.” (Bryman, 2016, p.283).

## Data Selection

Before performing a content analysis of the article titles, we want to make a selection of titles that is focused on the academic debate around migration, without any distracting non-topical titles that might obscure any topical shifts across the decades from 1951-2000.

Given the analysis above, we use the following three sets of articles for analysis:

1. main articles from REMP and IM, plus the IM review articles,
2. main articles from IMR
3. review articles from IMR

The reason to combine the main and review articles of IM is that the number of review articles is too low for independent content analysis. 

### References 

Bryman, A. (2016). Social research methods (Fifth edition.). Oxford ; New York: Oxford University Press.



In [275]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

# the name and location of the article records for the IM journal (in CSV format)
records_file = '../data/main-review-article-records.csv'

columns = ['article_title', 'issue_pub_year', 'publisher', 'article_type']
# load the csv data into a data frame
df = pd.read_csv(records_file, usecols=columns)
# show the first and last records of the dataset to demonstrate what the records look like
df.head(5)

,article_title,issue_pub_year,publisher,article_type
0,The importance of emigration for the solution ...,1951,REMP,main
1,European emigration overseas past and future,1951,REMP,main
2,Some aspects of migration problems in the Neth...,1952,REMP,main
3,Some quantitative aspects of future population...,1952,REMP,main
4,"The refugees as a burden, a stimulus, and a ch...",1951,REMP,main


In [276]:
def map_dataset(publisher, article_type):
    # all REMP and IM (published by Wiley) articles are bunlded in a single dataset
    if publisher == 'REMP' or publisher == 'Wiley':
        return 'REMP_IM'
    # The IMR articles are separated in review articles and research articles
    return 'IMR_research' if article_type == 'main' else 'IMR_review'

df['dataset'] = df.apply(lambda x: map_dataset(x['publisher'], x['article_type']), axis=1)
df.dataset.value_counts()

IMR_review      1842
IMR_research    1539
REMP_IM          901
Name: dataset, dtype: int64

### Analysing Title Words Per Decade

The articles are published over a period of several decades, and there might be shifts in the discourse over time. A next step is to group uni-grams and bi-grams per decade, to visualise shifts.

The first step is to group the article titles per decade. We derive the decade from the year that the issue was published. 

**TO DO: elaborate choice for per-decade.**

In [277]:
# adding a column per article with the publication decade
df['issue_decade'] = df.issue_pub_year.apply(lambda x: int(x/10) * 10 if not pd.isnull(x) else x)

df.issue_decade.value_counts()

1990    1490
1980    1236
1970    1052
1960     404
1950     100
Name: issue_decade, dtype: int64

In [278]:
df.groupby(['issue_decade', 'dataset']).size()

issue_decade  dataset     
1950          REMP_IM         100
1960          IMR_research    122
              IMR_review      129
              REMP_IM         153
1970          IMR_research    340
              IMR_review      592
              REMP_IM         120
1980          IMR_research    475
              IMR_review      519
              REMP_IM         242
1990          IMR_research    602
              IMR_review      602
              REMP_IM         286
dtype: int64

## Analysing Discourse Topics Through Publication Titles

The titles are at most only a highly condensed summary of the topics discussed in the publications, but nevertheless allows us to track what the debate focussed on and how attention shifted across the decades. 

We conduct a content analysis of the three sets of titles by comparing the frequencies of terms (e.g. individual words and common two-word phrases) across the datasets and across decades. Before counting terms, we normalise titles by removing punctuation, quote characters, superscript symbols for footnotes and stop words, then lowercasing all terms so that differences in case do not result in different terms.

In [279]:
import scripts.content_analysis as ca

# create a normalised version of the article title
# - remove footnote symbols, stopwords, punctuation and quote characters
# - standardise acronyms
# - remove 'Book Review: ' prefix from review article titles
df['normalised_title'] = df.article_title.apply(ca.normalise_title)
df.normalised_title

0       the importance of emigration for the solution ...
1            european emigration overseas past and future
2       some aspects of migration problems in the neth...
3       some quantitative aspects of future population...
4       the refugees as a burden a stimulus and a chal...
                              ...                        
4277                             chapter four going north
4278               chapter five the new york labor market
4279                   chapter six the settlement process
4280                             chapter seven conclusion
4281                                         bibliography
Name: normalised_title, Length: 4282, dtype: object

### Bigrams

An additional transformation step we take is identifying and marking common two-collocations so that they are treated as single terms. That is, a collocation like _United States_  should not be treated as two words _United_ and _States_ but as a single term.

Below we transform titles that have common collocations by replacing the whitespace between collocated words with an underscore, so they become a single term.

In [313]:
from scripts.content_analysis import mark_bigrams, extract_bigrams

titles = list(df.normalised_title)
bigrams = extract_bigrams(titles, stopwords)
df['bigrammed_title'] = df.normalised_title.apply(lambda x: mark_bigrams(x, bigrams))
df.bigrammed_title.head(5)

0    the importance of emigration for the solution ...
1         european emigration overseas past and future
2    some aspects of migration_problems in the neth...
3    some quantitative aspects of future population...
4    the refugees as a burden a stimulus and a chal...
Name: bigrammed_title, dtype: object

The third title has the collocation _migration_problems_. 


### From Terms to TF\*IDF

To compute TF\*IDF scores, we compute the TF and IDF parts separately on different representations of the titles:

- *IDF*: count the document frequencies and IDF of terms per title. This ensures that the IDF for a term is the same across the three datasets and across the decades, so that we can meaningfully interpret differences when comparing datasets and decades. 
- *TF*: the term frequency is computed on the aggregation of all titles per dataset, per decade. That is, we combine the titles for the publications in a single decade in a single dataset into a single document and count the frequency of each term.

We start with grouping publications titles per decade and dataset.

In [305]:
from collections import Counter
from collections import defaultdict

unigram_freq = defaultdict(dict)
grouped_titles = defaultdict(dict)

for decade in df.issue_decade.unique():
    for dataset in df.dataset.unique():
        titles = list(df[(df.issue_decade == decade) & (df.dataset == dataset)].bigrammed_title)
        grouped_titles[dataset][decade] = list(titles)


Next, we compute the IDF scores per term based on the individual titles as documents.

In [289]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

stopwords = ca.get_stopwords()


# make a list of all the titles with identified bigrams.
docs = list(df.bigrammed_title)

# instantiate a CountVectorizer() 
cv=CountVectorizer(stop_words=stopwords, analyzer='word') 
 
# compute word counts per title
word_count_vector=cv.fit_transform(docs)
# instantiate a tf*idf transformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
# fit the word counts per title to the tf*idf transformer
tfidf_transformer.fit(word_count_vector)


# extract the IDF values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# show the term with lowest and highest IDF scores
df_idf.sort_values(by=['idf_weights'])

,idf_weights
migration,3.112434
immigration,3.809449
united_states,3.829019
immigrants,3.865241
america,4.163912
...,...
hall,8.669262
halpine,8.669262
hamburg,8.669262
handicap,8.669262


Not surprisingly, terms like _migration_, _immigration_ and _immigrants_ have the lowest IDF scores as they are occur in more titles than other terms. Note that _united_states_ and _america_ also occur in many titles. 

The terms with the highest IDF scores are those that occur only in a single title.

The following step is merging the titles of all publications per decade and per dataset. That is, all titles of IMR Research publications in the 1970s are combined into a single text.

In [292]:
merged_titles_docs = []

# group publications by decade and dataset
for dataset in grouped_titles:
    for decade in grouped_titles[dataset]:
        # merged the titles of the grouped publication into a single document.
        merged_titles_doc = ' '.join(grouped_titles[dataset][decade])
        # add the merged title document to a dictionary for later usage
        merged_titles_docs.append(merged_titles_doc)

Finally, we compute the TF using the merged title documents

In [302]:
# count matrix 
count_vector=cv.transform(merged_titles_docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)
# the features are the individual terms
feature_names = cv.get_feature_names() 
# add a document title for each merged title document, based on the dataset label and the decade
text_titles = [f'{dataset}_{decade}' for dataset in grouped_titles for decade in grouped_titles[dataset]]

# finally, make a data frame with one column per decade per dataset and one row for each term in the collection
tfidf_df = pd.DataFrame(tf_idf_vector.T.toarray(), columns=text_titles, index=cv.get_feature_names())

# TO DO: Pick content terms as examples
tfidf_df.head(2)

,REMP_IM_1950,REMP_IM_1960,REMP_IM_1970,REMP_IM_1980,REMP_IM_1990,IMR_research_1950,IMR_research_1960,IMR_research_1970,IMR_research_1980,IMR_research_1990,IMR_review_1950,IMR_review_1960,IMR_review_1970,IMR_review_1980,IMR_review_1990
00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.011078,0.0,0.0
10,0.0,0.0,0.0,0.0,0.013705,0.0,0.061443,0.0,0.009722,0.03399,0.0,0.0,0.000000,0.0,0.0


### Title Analysis of REMP and IM Publications

Below, we show the top 20 highest TF\*IDF terms per decade for the REMP and IM publications:

In [310]:
top_n = 20

terms = defaultdict(list)
for decade in range(1950, 2000, 10):
    top_terms = tfidf_df[f'REMP_IM_{decade}'].sort_values(ascending=False).head(top_n)
    terms[decade] = list(top_terms[top_terms > 0.0].index)
    terms[decade] += ['' for i in range(top_n - len(terms[decade]))]
    
pd.DataFrame(terms)

,1950,1960,1970,1980,1990
0,immigration,migration,refugees,migration,migration
1,population,emigration,immigrants,migrants,emigration_dynamics
2,aspects,immigrants,cuban,social,australia
3,netherlands,assimilation,migration,origin,international_migration
4,emigration,study,integration,effects,migrants
5,migration,immigration,manpower,migrant_women,case
6,economic,development,united_states,canada,economic
7,dutch,australia,1959,situation,immigrants
8,refugee_problem,australian,australia,immigrants,japan
9,post,european,1968,countries,south


In terms of shifts over decades we see:

- The term 'Netherlands' is prominent in the 1950s, even though we removed the publications in the Dutch REMP series. The term 'Dutch' is still common in the 1960s, but from the 1970s onwards, neither term occurs in the top 20. 
- Australia is mentioned from the 1960s onwards and remains a common topic.
- Emigration is a top term in the 50s and 60s, immigration comes up in the 60s and remains a common topic. 
- Europe is a common topic in the 50s and 60s.

### Title Analysis of IMR Research Publications

Below, we show the top 20 highest TF\*IDF terms per decade for the IMR Research publications:

In [311]:
terms = defaultdict(list)
for decade in range(1950, 2000, 10):
    top_terms = tfidf_df[f'IMR_research_{decade}'].sort_values(ascending=False).head(top_n)
    terms[decade] = list(top_terms[top_terms > 0.0].index)
    terms[decade] += ['' for i in range(top_n - len(terms[decade]))]
    
pd.DataFrame(terms)

,1950,1960,1970,1980,1990
0,,migration,migration,migration,migration
1,,recent_books,united_states,international_newsletter,international_newsletter
2,,immigrants,immigration,united_states,immigration
3,,assimilation,international_newsletter,immigration,immigrants
4,,italian,italian,refugee,united_states
5,,recent,books_received,return_migration,new
6,,immigration,review,refugees,among
7,,international_migration_news,adjustment,international_migration,immigrant
8,,puerto_ricans,legislative,women,case
9,,ethnic,reviews,political,convention



- Italy: emigration to US is big policy issue
- US


### Title Analysis of IMR Review Publications

Below, we show the top 20 highest TF\*IDF terms per decade for the IMR Review publications:

In [312]:
terms = defaultdict(list)
for decade in range(1950, 2000, 10):
    top_terms = tfidf_df[f'IMR_review_{decade}'].sort_values(ascending=False).head(top_n)
    terms[decade] = list(top_terms[top_terms > 0.0].index)
    terms[decade] += ['' for i in range(top_n - len(terms[decade]))]
    
pd.DataFrame(terms)

,1950,1960,1970,1980,1990
0,,americans,america,america,america
1,,america,american,united_states,american
2,,spanish,americans,immigrants,immigration
3,,population,immigrants,ethnic,united_states
4,,jewish_community,history,migration,ethnicity
5,,italiana,migration,history,immigrants
6,,study,ethnic,ethnicity,migration
7,,immigrants,united_states,american,politics
8,,assimilation,study,der,life
9,,jewish,studies,emigration,identity


- more focus on ethnicity

**TO DO: discuss difference between book reviews and research articles. Are books for a different audience?**

### Visualisation Choices

We tried visualising the top N terms over time in a dynamic 'bar chart race', where the top N terms are selected for a 10 year period starting in 1950, then updating the top N by sliding the 10 year period forward by 1 year in each step. But this results in a very chaotic visualisation that is alost impossible to interpret. 

### Analysing Countries Mentioned in Titles

We map countries to continents, as larger geographic units.

In some titles, the nationality of a migrant group is mentioned, which differs from the country name. We add an analysis in which also nationalities are mapped to their respective country names and continents. E.g. `Polish` is mapped to `Poland` and `Europe`. 

We started with the countries and continents listed on the [World Atlas](https://www.worldatlas.com/cntycont.htm) website, and extended these several former countries (e.g. Soviet Union) and not formally-recognised countries (e.g. Kosovo, Palestina) as well as some larger regions (Caribbean, Latin America, Middle East). For the UK, we included England, Northern Ireland, Scotland and Wales, as these are also mentioned in some titles. 

For the nationalities, we used a list provided by Wikipedia ([List of adjectival and demonymic forms for countries and nations](https://en.wikipedia.org/w/index.php?title=List_of_adjectival_and_demonymic_forms_for_countries_and_nations&oldid=1004136953)).


### Countries and Nationalities in REMP and IM Publications

In [340]:
from scripts.countries import CountryLookup, show_counts

lookup = CountryLookup()
dataset = 'REMP_IM'
country_count = {}
continent_count = {}

for decade in grouped_titles[dataset]:
    titles = [title.replace('_', ' ') for title in grouped_titles[dataset][decade]]
    # First we count without nationalities
    country_counts, continent_counts = lookup.count_countries_continents(titles, include_nationalities=True)
    country_count[decade] = [f'{country} ({count})' if count > 0 else '' for country, count in country_counts.most_common(20)]
    continent_count[decade] = [f'{continent} ({count})' if count > 0 else '' for continent, count in continent_counts.most_common(20)]
    
country_df = pd.DataFrame(country_count)
country_df


,1950,1960,1970,1980,1990
0,Netherlands (14),Australia (23),United States (10),Canada (21),Australia (22)
1,Germany (9),Canada (7),Canada (8),United States (17),United States (16)
2,Australia (7),Netherlands (6),Cuba (7),Australia (10),Canada (14)
3,Brazil (4),Hungary (6),Australia (7),Israel (7),Turkey (12)
4,France (3),United States (5),Germany (6),Germany (6),Israel (11)
5,Austria (2),United Kingdom (5),New Zealand (4),Middle East (5),Japan (11)
6,Finland (2),Latin America (5),Greece (4),Greece (5),Mexico (10)
7,United States (2),Brazil (4),Latvia (3),Sweden (4),Greece (9)
8,Middle East (2),France (3),Italy (3),Spain (4),United Kingdom (5)
9,Italy (2),Italy (3),Israel (3),Netherlands (4),India (5)


In [339]:
continent_df = pd.DataFrame(continent_count)
continent_df


,1950,1960,1970,1980,1990
0,Europe (45),Europe (48),Europe (30),Europe (45),Asia (77)
1,Oceania (9),Oceania (24),North America (21),North America (42),Europe (41)
2,South America (5),North America (12),Asia (12),Asia (29),North America (40)
3,Asia (5),South America (10),Oceania (11),Oceania (11),Africa (27)
4,North America (2),Africa (5),South America (6),South America (10),Oceania (23)
5,,Asia (5),Africa (3),Africa (4),South America (8)


### Countries and Nationalities in IMR Research Publications

In [329]:
from scripts.countries import CountryLookup, show_counts

lookup = CountryLookup()
dataset = 'IMR_research'
country_count = {}
contintent_count = {}

for decade in grouped_titles[dataset]:
    titles = [title.replace('_', ' ') for title in grouped_titles[dataset][decade]]
    # First we count without nationalities
    country_counts, continent_counts = lookup.count_countries_continents(titles, include_nationalities=True)
    country_count[decade] = [f'{country} ({count})' if count > 0 else '' for country, count in country_counts.most_common(20)]
    continent_count[decade] = continent_counts
    
country_df = pd.DataFrame(country_count)
country_df

,1950,1960,1970,1980,1990
0,,United States (13),United States (58),United States (47),United States (66)
1,,Italy (13),Italy (28),Mexico (24),Italy (21)
2,,Puerto Rico (7),Mexico (18),Canada (18),Canada (20)
3,,Canada (6),Greece (10),Australia (14),Mexico (20)
4,,Australia (5),Australia (8),Germany (10),Australia (18)
5,,United Kingdom (2),Canada (7),Dominican Republic (8),Germany (12)
6,,Mexico (2),Philippines (6),Dominica (8),Puerto Rico (11)
7,,Israel (2),Latin America (6),Vietnam (8),China (10)
8,,Brazil (2),Puerto Rico (4),France (7),Israel (9)
9,,Japan (2),Dominican Republic (3),Nigeria (6),Turkey (8)


### Countries and Nationalities in IMR Review Publications

In [331]:
from scripts.countries import CountryLookup, show_counts

lookup = CountryLookup()
dataset = 'IMR_review'
country_count = {}
contintent_count = {}

for decade in grouped_titles[dataset]:
    titles = [title.replace('_', ' ') for title in grouped_titles[dataset][decade]]
    # First we count without nationalities
    country_counts, continent_counts = lookup.count_countries_continents(titles, include_nationalities=True)
    country_count[decade] = [f'{country} ({count})' if count > 0 else '' for country, count in country_counts.most_common(20)]
    continent_count[decade] = continent_counts
    
country_df = pd.DataFrame(country_count)
country_df

,1950,1960,1970,1980,1990
0,,United States (19),United States (135),United States (96),United States (140)
1,,Italy (8),Mexico (25),Italy (28),Mexico (31)
2,,Australia (7),Canada (21),Canada (23),China (29)
3,,Puerto Rico (5),Italy (21),Mexico (16),Italy (21)
4,,Spain (4),Germany (16),Germany (16),Macau (20)
5,,China (4),China (13),China (13),Germany (17)
6,,Ireland (3),Macau (13),Australia (12),Canada (15)
7,,Mexico (3),Puerto Rico (12),Ireland (11),France (13)
8,,Macau (3),India (12),Macau (11),Korea (12)
9,,England (3),Poland (11),Japan (11),Caribbean (11)
